In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.4.2
2.8.2


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation


def sum_over(*args):
    return sum(args)

In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
CategoricalParameter('A.1_Brate', [0, 1, 2])
CategoricalParameter('A.2_Brate', [0, 1, 2])
CategoricalParameter('A.3_Brate', [0, 1, 2])
CategoricalParameter('A.4_Brate', [0, 1, 2])
CategoricalParameter('A.5_Brate', [0, 1, 2])


In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage 0', 'A.1_Expected Annual Damage 1', 'A.1_Expected Annual Damage 2', 'A.1_Dike Investment Costs 0', 'A.1_Dike Investment Costs 1', 'A.1_Dike Investment Costs 2'), function=<function sum_over at 0x0000012ED2B21310>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths 0', 'A.1_Expected Number of Deaths 1', 'A.1_Expected Number of Deaths 2'), function=<function sum_over at 0x0000012ED2B21310>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage 0', 'A.2_Expected Annual Damage 1', 'A.2_Expected Annual Damage 2', 'A.2_Dike Investment Costs 0', 'A.2_Dike Investment Costs 1', 'A.2_Dike Investment Costs 2'), function=<function sum_over at 0x0000012ED2B21310>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths 0', 'A.2_Expected Number of Deaths 1', 'A.2_Expected Number of Deaths 2'), function=<function sum_over at 0

In [8]:
# defining specific policies
# for example, policy 1 is about extra protection in upper boundary
# policy 2 is about extra protection in lower boundary
# policy 3 is extra protection in random locations


def get_0_dict():
    return {l.name: 0 for l in dike_model.levers}


policies = [
    Policy(
        "policy 1",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 2",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 0, "A.3_DikeIncrease 0": 5})
    ),
    Policy(
        "policy 3",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 0, "A.3_DikeIncrease 0": 10})
    ),
    Policy(
        "policy 4",
        **dict(get_0_dict(), **{"0_RfR 0": 1, "0_RfR 1": 0, "0_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 5",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 1, "0_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 6",
        **dict(get_0_dict(), **{"0_RfR 0": 0, "0_RfR 1": 0, "0_RfR 2": 1, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 7",
        **dict(get_0_dict(), **{"1_RfR 0": 1, "1_RfR 1": 0, "1_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 8",
        **dict(get_0_dict(), **{"1_RfR 0": 0, "1_RfR 1": 1, "1_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 9",
        **dict(get_0_dict(), **{"1_RfR 0": 0, "1_RfR 1": 0, "1_RfR 2": 1, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 10",
        **dict(get_0_dict(), **{"2_RfR 0": 1, "2_RfR 1": 0, "2_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 11",
        **dict(get_0_dict(), **{"2_RfR 0": 1, "2_RfR 1": 0, "2_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
    Policy(
        "policy 12",
        **dict(get_0_dict(), **{"2_RfR 0": 1, "2_RfR 1": 0, "2_RfR 2": 0, "A.3_DikeIncrease 0": 0})
    ),
]

In [9]:
# True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle1 = True

if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)

else:
    # pass the policies list to EMA workbench experiment runs
    n_scenarios = 25
    with MultiprocessingEvaluator(dike_model) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, policies)

    # Save results in Pickle file
    with open('data/formulation_results.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)

[MainProcess/INFO] pool started with 8 workers
[MainProcess/INFO] performing 25 scenarios * 12 policies * 1 model(s) = 300 experiments
100%|████████████████████████████████████████| 300/300 [04:53<00:00,  1.02it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


In [10]:
import pandas as pd

a, b = results
df = pd.DataFrame(b)
df["policy"] = pd.DataFrame(a)["policy"]
df

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,policy
0,0.000000e+00,0.000000,3.360490e+06,0.003507,4.170229e+07,0.080676,7.712770e+07,0.037279,0.000000e+00,0.000000,0.0,0.0,policy 1
1,1.265051e+09,0.954712,0.000000e+00,0.000000,1.060916e+09,2.009208,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,policy 1
2,0.000000e+00,0.000000,1.968211e+07,0.022936,3.744110e+07,0.081634,0.000000e+00,0.000000,1.225323e+08,0.133857,0.0,0.0,policy 1
3,4.557337e+06,0.003119,2.543544e+08,0.218760,0.000000e+00,0.000000,1.069845e+07,0.004603,0.000000e+00,0.000000,0.0,0.0,policy 1
4,1.959312e+09,1.671697,0.000000e+00,0.000000,8.172553e+06,0.018682,0.000000e+00,0.000000,0.000000e+00,0.000000,0.0,0.0,policy 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3.469688e+09,2.004698,2.416862e+08,0.184684,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,30700000.0,0.0,policy 12
296,0.000000e+00,0.000000,5.468757e+06,0.006692,1.886993e+07,0.043029,1.044842e+08,0.056928,6.088358e+08,0.631826,30700000.0,0.0,policy 12
297,6.224432e+08,0.475863,9.049086e+08,0.843249,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,30700000.0,0.0,policy 12
298,0.000000e+00,0.000000,9.715568e+06,0.012186,3.351533e+07,0.078183,1.299877e+08,0.067307,0.000000e+00,0.000000,30700000.0,0.0,policy 12


In [11]:
def q05(x):
    return x.quantile(0.05)


def q95(x):
    return x.quantile(0.95)

In [12]:
group_df = df.groupby(by="policy")
agg_par = ["mean", "min", "max", q05, q95]
agg_dict = {
    "A.3 Total Costs": agg_par,
    "A.3_Expected Number of Deaths": agg_par,
    "RfR Total Costs": agg_par,
    "Expected Evacuation Costs": agg_par,
}
TCA_df = group_df.agg(agg_dict)
TCA_df

A.3 Total Costs                                            \
                     mean           min           max           q05   
policy                                                                
policy 1     4.027475e+08  0.000000e+00  2.482597e+09  0.000000e+00   
policy 10    2.452492e+08  0.000000e+00  2.371300e+09  0.000000e+00   
policy 11    2.452492e+08  0.000000e+00  2.371300e+09  0.000000e+00   
policy 12    2.452492e+08  0.000000e+00  2.371300e+09  0.000000e+00   
policy 2     4.398000e+07  2.879840e+07  3.881443e+08  2.879840e+07   
policy 3     4.421502e+07  4.421502e+07  4.421502e+07  4.421502e+07   
policy 4     5.036353e+08  0.000000e+00  2.484053e+09  0.000000e+00   
policy 5     5.036353e+08  0.000000e+00  2.484053e+09  0.000000e+00   
policy 6     5.036353e+08  0.000000e+00  2.484053e+09  0.000000e+00   
policy 7     5.023287e+08  0.000000e+00  2.484053e+09  0.000000e+00   
policy 8     5.023287e+08  0.000000e+00  2.484053e+09  0.000000e+00   
policy 9     5.023287e+08  0.000000e+00  2.484053e+09  0.000000e+00   

                        A.3_Expected Number of Deaths                      \
                    q95                          mean  min       max  q05   
policy                                                                      
policy 1   2.115484e+09                      0.709059  0.0  3.707753  0.0   
policy 10  1.527570e+09                      0.418578  0.0  3.432326  0.0   
policy 11  1.527570e+09                      0.418578  0.0  3.432326  0.0   
policy 12  1.527570e+09                      0.418578  0.0  3.432326  0.0   
policy 2   4.495372e+07                      0.033906  0.0  0.815647  0.0   
policy 3   4.421502e+07                      0.000000  0.0  0.000000  0.0   
policy 4   2.115484e+09                      0.874568  0.0  3.707753  0.0   
policy 5   2.115484e+09                      0.874568  0.0  3.707753  0.0   
policy 6   2.115484e+09                      0.874568  0.0  3.707753  0.0   
policy 7   2.115484e+09                      0.872382  0.0  3.707753  0.0   
policy 8   2.115484e+09                      0.872382  0.0  3.707753  0.0   
policy 9   2.115484e+09                      0.872382  0.0  3.707753  0.0   

                    RfR Total Costs                                         \
                q95            mean          min          max          q05   
policy                                                                       
policy 1   3.652713             0.0          0.0          0.0          0.0   
policy 10  3.093675      30700000.0   30700000.0   30700000.0   30700000.0   
policy 11  3.093675      30700000.0   30700000.0   30700000.0   30700000.0   
policy 12  3.093675      30700000.0   30700000.0   30700000.0   30700000.0   
policy 2   0.025596             0.0          0.0          0.0          0.0   
policy 3   0.000000             0.0          0.0          0.0          0.0   
policy 4   3.655533      84600000.0   84600000.0   84600000.0   84600000.0   
policy 5   3.655533      84600000.0   84600000.0   84600000.0   84600000.0   
policy 6   3.655533      84600000.0   84600000.0   84600000.0   84600000.0   
policy 7   3.655533     217800000.0  217800000.0  217800000.0  217800000.0   
policy 8   3.655533     217800000.0  217800000.0  217800000.0  217800000.0   
policy 9   3.655533     217800000.0  217800000.0  217800000.0  217800000.0   

                       Expected Evacuation Costs                      
                   q95                      mean  min  max  q05  q95  
policy                                                                
policy 1           0.0                       0.0  0.0  0.0  0.0  0.0  
policy 10   30700000.0                       0.0  0.0  0.0  0.0  0.0  
policy 11   30700000.0                       0.0  0.0  0.0  0.0  0.0  
policy 12   30700000.0                       0.0  0.0  0.0  0.0  0.0  
policy 2           0.0                       0.0  0.0  0.0  0.0  0.0  
policy 3           0.0                       0.0 

In [13]:
TCA_df.to_csv("results/TCA3_results.csv")